In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [3]:
# Initializations
init_x_11 = np.array([0.2484566697970932,0.7515433302029068])
init_x_12 = np.array([0.5000099165391315,0.4999900834608686])
init_x_21 = init_x_12
init_x_22 = np.array([0.7515267825525722, 0.24847321744742773])
init_y_11 = np.array([0.24846816041770087, 0.7515318395822992])
init_y_12 = np.array([0.16754754156194188, 0.8324524584380582])
init_y_21 = init_y_12
init_y_22 = np.array([0.24847672933953846,0.7515232706604615])
init_x = np.array([0.3,0.7])
init_y = np.array([0.7,0.3])

#T_lst = [1e3,3e3, 1e4]
T_lst = [1e3, 3e3, 1e4, 3e4, 1e5, 3e5, 1e6, 3e6, 1e7]

In [4]:
# ------------------------------ Multiple Trial Figure (Figure 1(a)) Data --------------------------------------

In [4]:
def prop_exponential(r):
    x = np.exp(r-np.mean(r))
    x = x/sum(x)
    return x

def NE_gap(x_11,y_11,x_12,y_12,x_21,y_21,x_22,y_22,x,y,R):
    #R = np.identity(2)
    qx_11 = np.dot(R,y_11)
    qx_12 = np.dot(R,y_12)
    qx_21 = np.dot(R,y_21)
    qx_22 = np.dot(R,y_22)
    qy_11 = np.dot(np.transpose(R),x_11)
    qy_12 = np.dot(np.transpose(R),x_12)
    qy_21 = np.dot(np.transpose(R),x_21)
    qy_22 = np.dot(np.transpose(R),x_22)
    
    r_11 = np.max(qx_11)
    r_12 = np.max(qx_12)
    r_21 = np.max(qx_21)
    r_22 = np.max(qx_22)
    Qx = (0.1*R + np.array([[r_11,r_12],[r_21,r_22]]))
    qx = np.dot(Qx,y)
    
    r_11 = np.min(qy_11)
    r_12 = np.min(qy_12)
    r_21 = np.min(qy_21)
    r_22 = np.min(qy_22)
    Qy = (0.1*R + np.array([[r_11,r_12],[r_21,r_22]]))
    qy = np.dot(np.transpose(Qy),x)
    return np.max(qx) - np.min(qy)
def update_xy(x,y,R,lr):
    gx = np.dot(R,y)
    gy = -np.dot(np.transpose(R),x)
    x = x * np.exp(gx*lr)
    y = y * np.exp(gy*lr)
    x = x / sum(x)
    y = y / sum(y)
    r = np.dot(x, np.dot(R,y))
    return x,y,r

In [6]:
# FTRL
H = 2
R = np.identity(2)
data_lst = []
NE_gap_lst = []
dist_lst = []
for T in T_lst:
    T = int(T)
    lr = 10/np.sqrt(T)
    Q_11 = np.ones([2,2])
    Q_12 = np.ones([2,2])
    Q_21 = np.ones([2,2])
    Q_22 = np.ones([2,2])
    Q = 2*np.ones([2,2])

    qx_11 = 0#np.dot(Q_11,y_11)
    qy_11 = 0#-np.dot(np.transpose(Q_11),x_11)
    qx_12 = 0#np.dot(Q_12,y_12)
    qy_12 = 0#-np.dot(np.transpose(Q_12),x_12)
    qx_21 = 0#np.dot(Q_21,y_21)
    qy_21 = 0#-np.dot(np.transpose(Q_21),x_21)
    qx_22 = 0#np.dot(Q_22,y_22)
    qy_22 = 0#-np.dot(np.transpose(Q_22),x_22)
    qx = 0#np.dot(Q,y)
    qy = 0#-np.dot(np.transpose(Q),x)
    w_t = 0

    output_x_11 = 0
    output_y_11 = 0
    output_x_12 = 0
    output_y_12 = 0
    output_x_21 = 0
    output_y_21 = 0
    output_x_22 = 0
    output_y_22 =0 
    output_x = 0
    output_y = 0

    for t in range(1,T+1):
        # Actor
        if w_t == 0:
            eta = 0
        else:
            eta = lr/w_t
        x_11 = prop_exponential(eta*qx_11 + np.log(init_x_11))
        y_11 = prop_exponential(eta*qy_11+ np.log(init_y_11))
        x_12 = prop_exponential(eta*qx_12+ np.log(init_x_12))
        y_12 = prop_exponential(eta*qy_12+ np.log(init_y_12))
        x_21 = prop_exponential(eta*qx_21+ np.log(init_x_21))
        y_21 = prop_exponential(eta*qy_21+ np.log(init_y_21))
        x_22 = prop_exponential(eta*qx_22+ np.log(init_x_22))
        y_22 = prop_exponential(eta*qy_22+ np.log(init_y_22))
        x = prop_exponential(eta*qx+ np.log(init_x))
        y = prop_exponential(eta*qy+ np.log(init_y))


        # Critic 
        alpha_t = (H+1)/(H+t) 
        if t == 1:
            alpha_t_1 = 1
        else:
            alpha_t_1 = alpha_t_1*(1-alpha_t)
        w_t = alpha_t/alpha_t_1
        r_11 = np.dot(x_11, np.dot(R,y_11))
        r_12 = np.dot(x_12, np.dot(R,y_12))
        r_21 = np.dot(x_21, np.dot(R,y_21))
        r_22 = np.dot(x_22, np.dot(R,y_22))
        Q = (1-alpha_t)*Q + alpha_t*(0.1*R + np.array([[r_11,r_12],[r_21,r_22]]))

        qx_11 = qx_11 + w_t* np.dot(R,y_11)
        qy_11 = qy_11 - w_t*np.dot(np.transpose(R),x_11)
        qx_12 = qx_12 + w_t* np.dot(R,y_12)
        qy_12 = qy_12 - w_t*np.dot(np.transpose(R),x_12)
        qx_21 = qx_21 + w_t* np.dot(R,y_21)
        qy_21 = qy_21 - w_t*np.dot(np.transpose(R),x_21)
        qx_22 = qx_22 + w_t* np.dot(R,y_22)
        qy_22 = qy_22 - w_t*np.dot(np.transpose(R),x_22)
        qx = qx + w_t*np.dot(Q,y)
        qy = qy - w_t*np.dot(np.transpose(Q),x)

        # Output policy
        output_x_11 = (1-alpha_t)*output_x_11 + alpha_t*x_11
        output_y_11 = (1-alpha_t)*output_y_11 + alpha_t*y_11
        output_x_12 = (1-alpha_t)*output_x_12 + alpha_t*x_12
        output_y_12 = (1-alpha_t)*output_y_12 + alpha_t*y_12
        output_x_21 = (1-alpha_t)*output_x_21 + alpha_t*x_21
        output_y_21 = (1-alpha_t)*output_y_21 + alpha_t*y_21
        output_x_22 = (1-alpha_t)*output_x_22 + alpha_t*x_22
        output_y_22 = (1-alpha_t)*output_y_22 + alpha_t*y_22
        output_x = (1-alpha_t)*output_x + alpha_t*x
        output_y = (1-alpha_t)*output_y + alpha_t*y
    data_lst.append([output_x_11,output_y_11,output_x_12,output_y_12,
                    output_x_21,output_y_21,output_x_22,output_y_22,output_x,output_y])
    NE_gap_val = NE_gap(output_x_11,output_y_11,output_x_12,output_y_12,
                    output_x_21,output_y_21,output_x_22,output_y_22,output_x,output_y,R)
    NE_gap_lst.append(NE_gap_val)
    dist_lst.append(np.sum(np.abs(output_x_11-0.5) + np.abs(output_y_11-0.5) +
                          np.abs(output_x_12-0.5) + np.abs(output_y_12-0.5) +
                           np.abs(output_x_21-0.5) + np.abs(output_y_21-0.5) +
                           np.abs(output_x_22-0.5) + np.abs(output_y_22-0.5) +
                           np.abs(output_x-0.5) + np.abs(output_y-0.5)
                          ))
    print('T = ', T, '  NE_gap:', NE_gap_val)
    if T>= 1e6-2:
        np.savez('FTRL', T_lst = T_lst, NE_gap_lst = NE_gap_lst, data_lst = data_lst, dist_lst = dist_lst)
np.savez('FTRL', T_lst = T_lst, NE_gap_lst = NE_gap_lst, data_lst = data_lst, dist_lst = dist_lst)
#np.savez('FTRL', T_lst = T_lst, NE_gap_lst = NE_gap_lst, data_lst = data_lst, dist_lst = dist_lst)
#np.savez('FTRL_long_run', T_lst = T_lst, NE_gap_lst = NE_gap_lst, data_lst = data_lst, dist_lst = dist_lst)

T =  1000   NE_gap: 0.030737141941136192
T =  3000   NE_gap: 0.01945493359062933
T =  10000   NE_gap: 0.005715018220575407
T =  30000   NE_gap: 0.004315426323948213
T =  100000   NE_gap: 0.0017109848503203962
T =  300000   NE_gap: 0.0009732743322863335
T =  1000000   NE_gap: 0.0005351263441414345
T =  3000000   NE_gap: 0.00031425661688933015
T =  10000000   NE_gap: 0.0001709820541704321


In [6]:
# OFTRL
H = 2
R = np.identity(2)
data_lst = []
NE_gap_lst = []
dist_lst = []
for T in T_lst:
    T = int(T)
    lr = 1/np.power(T, 1/6)
    Q_11 = R#np.ones([2,2])
    Q_12 = R#np.ones([2,2])
    Q_21 = R#np.ones([2,2])
    Q_22 = R#np.ones([2,2])
    Q = np.ones([2,2])

    qx_11 = 0
    qy_11 = 0
    qx_12 = 0
    qy_12 = 0
    qx_21 = 0
    qy_21 = 0
    qx_22 = 0
    qy_22 = 0
    qx = 0
    qy = 0
    lx_11 = 0
    ly_11 = 0
    lx_12 = 0
    ly_12 = 0
    lx_21 = 0
    ly_21 = 0
    lx_22 = 0
    ly_22 = 0
    lx = 0
    ly = 0

    output_x_11 = 0
    output_y_11 = 0
    output_x_12 = 0
    output_y_12 = 0
    output_x_21 = 0
    output_y_21 = 0
    output_x_22 = 0
    output_y_22 =0 
    output_x = 0
    output_y = 0
    
    w_t = 0
    for t in range(1,T+1):
        
        if w_t == 0:
            eta = 0
        # Actor
        x_11 = prop_exponential(eta*(qx_11 + w_t*lx_11) + np.log(init_x_11))
        y_11 = prop_exponential(eta*(qy_11+ w_t*ly_11) + np.log(init_y_11))
        x_12 = prop_exponential(eta*(qx_12+ w_t*lx_12) + np.log(init_x_12))
        y_12 = prop_exponential(eta*(qy_12+ w_t*ly_12) + np.log(init_y_12))
        x_21 = prop_exponential(eta*(qx_21+ w_t*lx_21) + np.log(init_x_21))
        y_21 = prop_exponential(eta*(qy_21+ w_t*ly_21) + np.log(init_y_21))
        x_22 = prop_exponential(eta*(qx_22+ w_t*lx_22) + np.log(init_x_22))
        y_22 = prop_exponential(eta*(qy_22+ w_t*ly_22) + np.log(init_y_22))
        x = prop_exponential(eta*(qx+ w_t*lx) + np.log(init_x))
        y = prop_exponential(eta*(qy+ w_t*ly)+ np.log(init_y))
        
        alpha_t = (H+1)/(H+t) 
        if t == 1:
            alpha_t_1 = 1
        else:
            alpha_t_1 = alpha_t_1*(1-alpha_t)
        w_t = alpha_t/alpha_t_1
        eta = lr/w_t


        # Critic 

        r_11 = np.dot(x_11, np.dot(R,y_11))
        r_12 = np.dot(x_12, np.dot(R,y_12))
        r_21 = np.dot(x_21, np.dot(R,y_21))
        r_22 = np.dot(x_22, np.dot(R,y_22))
        Q = (1-alpha_t)*Q + alpha_t*(0.1*R + np.array([[r_11,r_12],[r_21,r_22]]))

        lx_11 = np.dot(R,y_11)
        ly_11 = -np.dot(np.transpose(R),x_11)
        lx_12 = np.dot(R,y_12)
        ly_12 = -np.dot(np.transpose(R),x_12)
        lx_21 = np.dot(R,y_21)
        ly_21 = -np.dot(np.transpose(R),x_21)
        lx_22 = np.dot(R,y_22)
        ly_22 = -np.dot(np.transpose(R),x_22)
        lx = np.dot(Q,y)
        ly = -np.dot(np.transpose(Q),x)
        qx_11 = qx_11 + w_t* lx_11
        qy_11 = qy_11 + w_t*ly_11
        qx_12 = qx_12 + w_t* lx_12
        qy_12 = qy_12 + w_t*ly_12
        qx_21 = qx_21 + w_t* lx_21
        qy_21 = qy_21 + w_t*ly_21
        qx_22 = qx_22 + w_t* lx_22
        qy_22 = qy_22 + w_t*ly_22
        qx = qx + w_t*lx
        qy = qy + w_t*ly

        # Output policy
        output_x_11 = (1-alpha_t)*output_x_11 + alpha_t*x_11
        output_y_11 = (1-alpha_t)*output_y_11 + alpha_t*y_11
        output_x_12 = (1-alpha_t)*output_x_12 + alpha_t*x_12
        output_y_12 = (1-alpha_t)*output_y_12 + alpha_t*y_12
        output_x_21 = (1-alpha_t)*output_x_21 + alpha_t*x_21
        output_y_21 = (1-alpha_t)*output_y_21 + alpha_t*y_21
        output_x_22 = (1-alpha_t)*output_x_22 + alpha_t*x_22
        output_y_22 = (1-alpha_t)*output_y_22 + alpha_t*y_22
        output_x = (1-alpha_t)*output_x + alpha_t*x
        output_y = (1-alpha_t)*output_y + alpha_t*y
    data_lst.append([output_x_11,output_y_11,output_x_12,output_y_12,
                    output_x_21,output_y_21,output_x_22,output_y_22,output_x,output_y])
    NE_gap_val = NE_gap(output_x_11,output_y_11,output_x_12,output_y_12,
                    output_x_21,output_y_21,output_x_22,output_y_22,output_x,output_y,R)
    NE_gap_lst.append(NE_gap_val)
    dist_lst.append(np.sum(np.abs(output_x_11-0.5) + np.abs(output_y_11-0.5) +
                          np.abs(output_x_12-0.5) + np.abs(output_y_12-0.5) +
                           np.abs(output_x_21-0.5) + np.abs(output_y_21-0.5) +
                           np.abs(output_x_22-0.5) + np.abs(output_y_22-0.5) +
                           np.abs(output_x-0.5) + np.abs(output_y-0.5)
                          ))
    print('T = ', T, '  NE_gap:', NE_gap_val)
    if T >= 1e6-1:
        np.savez('OFTRL', T_lst = T_lst, NE_gap_lst = NE_gap_lst,data_lst = data_lst, dist_lst = dist_lst)
np.savez('OFTRL', T_lst = T_lst, NE_gap_lst = NE_gap_lst,data_lst = data_lst, dist_lst = dist_lst)

T =  1000   NE_gap: 0.017364315684939013
T =  3000   NE_gap: 0.006908679346115942
T =  10000   NE_gap: 0.0025183469643927925
T =  30000   NE_gap: 0.0010057992275074312
T =  100000   NE_gap: 0.0003684220017639017
T =  300000   NE_gap: 0.00014743151035701008
T =  1000000   NE_gap: 5.40495098484417e-05
T =  3000000   NE_gap: 2.1635524308671883e-05
T =  10000000   NE_gap: 7.932804384602754e-06


In [ ]:
# OFTRL constant learning rate
H = 2
R = np.identity(2)
data_lst = []
NE_gap_lst = []
dist_lst = []
for T in T_lst:
    T = int(T)
    #lr = 1/np.power(T, 1/6)
    lr = 1 # try constant learning rate to see if it can achieve O(1/T) convergence
    Q_11 = R#np.ones([2,2])
    Q_12 = R#np.ones([2,2])
    Q_21 = R#np.ones([2,2])
    Q_22 = R#np.ones([2,2])
    Q = np.ones([2,2])

    qx_11 = 0
    qy_11 = 0
    qx_12 = 0
    qy_12 = 0
    qx_21 = 0
    qy_21 = 0
    qx_22 = 0
    qy_22 = 0
    qx = 0
    qy = 0
    lx_11 = 0
    ly_11 = 0
    lx_12 = 0
    ly_12 = 0
    lx_21 = 0
    ly_21 = 0
    lx_22 = 0
    ly_22 = 0
    lx = 0
    ly = 0

    output_x_11 = 0
    output_y_11 = 0
    output_x_12 = 0
    output_y_12 = 0
    output_x_21 = 0
    output_y_21 = 0
    output_x_22 = 0
    output_y_22 =0 
    output_x = 0
    output_y = 0
    
    w_t = 0
    for t in range(1,T+1):
        
        if w_t == 0:
            eta = 0
        # Actor
        x_11 = prop_exponential(eta*(qx_11 + w_t*lx_11) + np.log(init_x_11))
        y_11 = prop_exponential(eta*(qy_11+ w_t*ly_11) + np.log(init_y_11))
        x_12 = prop_exponential(eta*(qx_12+ w_t*lx_12) + np.log(init_x_12))
        y_12 = prop_exponential(eta*(qy_12+ w_t*ly_12) + np.log(init_y_12))
        x_21 = prop_exponential(eta*(qx_21+ w_t*lx_21) + np.log(init_x_21))
        y_21 = prop_exponential(eta*(qy_21+ w_t*ly_21) + np.log(init_y_21))
        x_22 = prop_exponential(eta*(qx_22+ w_t*lx_22) + np.log(init_x_22))
        y_22 = prop_exponential(eta*(qy_22+ w_t*ly_22) + np.log(init_y_22))
        x = prop_exponential(eta*(qx+ w_t*lx) + np.log(init_x))
        y = prop_exponential(eta*(qy+ w_t*ly)+ np.log(init_y))
        
        alpha_t = (H+1)/(H+t) 
        if t == 1:
            alpha_t_1 = 1
        else:
            alpha_t_1 = alpha_t_1*(1-alpha_t)
        w_t = alpha_t/alpha_t_1
        eta = lr/w_t


        # Critic 

        r_11 = np.dot(x_11, np.dot(R,y_11))
        r_12 = np.dot(x_12, np.dot(R,y_12))
        r_21 = np.dot(x_21, np.dot(R,y_21))
        r_22 = np.dot(x_22, np.dot(R,y_22))
        Q = (1-alpha_t)*Q + alpha_t*(0.1*R + np.array([[r_11,r_12],[r_21,r_22]]))

        lx_11 = np.dot(R,y_11)
        ly_11 = -np.dot(np.transpose(R),x_11)
        lx_12 = np.dot(R,y_12)
        ly_12 = -np.dot(np.transpose(R),x_12)
        lx_21 = np.dot(R,y_21)
        ly_21 = -np.dot(np.transpose(R),x_21)
        lx_22 = np.dot(R,y_22)
        ly_22 = -np.dot(np.transpose(R),x_22)
        lx = np.dot(Q,y)
        ly = -np.dot(np.transpose(Q),x)
        qx_11 = qx_11 + w_t* lx_11
        qy_11 = qy_11 + w_t*ly_11
        qx_12 = qx_12 + w_t* lx_12
        qy_12 = qy_12 + w_t*ly_12
        qx_21 = qx_21 + w_t* lx_21
        qy_21 = qy_21 + w_t*ly_21
        qx_22 = qx_22 + w_t* lx_22
        qy_22 = qy_22 + w_t*ly_22
        qx = qx + w_t*lx
        qy = qy + w_t*ly

        # Output policy
        output_x_11 = (1-alpha_t)*output_x_11 + alpha_t*x_11
        output_y_11 = (1-alpha_t)*output_y_11 + alpha_t*y_11
        output_x_12 = (1-alpha_t)*output_x_12 + alpha_t*x_12
        output_y_12 = (1-alpha_t)*output_y_12 + alpha_t*y_12
        output_x_21 = (1-alpha_t)*output_x_21 + alpha_t*x_21
        output_y_21 = (1-alpha_t)*output_y_21 + alpha_t*y_21
        output_x_22 = (1-alpha_t)*output_x_22 + alpha_t*x_22
        output_y_22 = (1-alpha_t)*output_y_22 + alpha_t*y_22
        output_x = (1-alpha_t)*output_x + alpha_t*x
        output_y = (1-alpha_t)*output_y + alpha_t*y
    data_lst.append([output_x_11,output_y_11,output_x_12,output_y_12,
                    output_x_21,output_y_21,output_x_22,output_y_22,output_x,output_y])
    NE_gap_val = NE_gap(output_x_11,output_y_11,output_x_12,output_y_12,
                    output_x_21,output_y_21,output_x_22,output_y_22,output_x,output_y,R)
    NE_gap_lst.append(NE_gap_val)
    dist_lst.append(np.sum(np.abs(output_x_11-0.5) + np.abs(output_y_11-0.5) +
                          np.abs(output_x_12-0.5) + np.abs(output_y_12-0.5) +
                           np.abs(output_x_21-0.5) + np.abs(output_y_21-0.5) +
                           np.abs(output_x_22-0.5) + np.abs(output_y_22-0.5) +
                           np.abs(output_x-0.5) + np.abs(output_y-0.5)
                          ))
    print('T = ', T, '  NE_gap:', NE_gap_val)
    if T >= 1e6-1:
        np.savez('OFTRL_const_lr', T_lst = T_lst, NE_gap_lst = NE_gap_lst,data_lst = data_lst, dist_lst = dist_lst)
        #np.savez('OFTRL', T_lst = T_lst, NE_gap_lst = NE_gap_lst,data_lst = data_lst, dist_lst = dist_lst)
np.savez('OFTRL_const_lr', T_lst = T_lst, NE_gap_lst = NE_gap_lst,data_lst = data_lst, dist_lst = dist_lst)
#np.savez('OFTRL', T_lst = T_lst, NE_gap_lst = NE_gap_lst,data_lst = data_lst, dist_lst = dist_lst)

In [12]:
## Counter example
def update_xy(x,y,R,lr):
    gx = np.dot(R,y)
    gy = -np.dot(np.transpose(R),x)
    x = x * np.exp(gx*lr)
    y = y * np.exp(gy*lr)
    x = x / sum(x)
    y = y / sum(y)
    r = np.dot(x, np.dot(R,y))
    return x,y,r
data_lst = []
NE_gap_lst = []
dist_lst = []
R = np.identity(2)

for T in T_lst:
    T = int(T)
    lr = 1/np.sqrt(T)
    #lr = 0.1
    # ------------------------------------------------------------------------------------------
    # Running MD-critic for given the following initialization
    
    # initialization H=1
    x = init_x
    y = init_y

    # initialization on H=2
    x_11 = init_x_11#np.array([.25,.75]) #x_slice[:,idx_max]
    x_12 = init_x_12#np.array([.5,.5])#x_slice[:,idx_mid]
    x_21 = init_x_21#np.array([.5,.5])#x_slice[:,idx_mid]
    x_22 = init_x_22#np.array([.75,.25])#x_slice[:,idx_min]
    y_11 = init_y_11#np.array([.25,.75])#y_slice[:,idx_max]
    y_12 = init_y_12#np.array([.17,.83])#y_slice[:,idx_mid]
    y_21 = init_y_21#np.array([.17,.83])#y_slice[:,idx_mid]
    y_22 = init_y_22#np.array([.25,.75])#y_slice[:,idx_min]
    
    output_x_11 = 0
    output_y_11 = 0
    output_x_12 = 0
    output_y_12 = 0
    output_x_21 = 0
    output_y_21 = 0
    output_x_22 = 0
    output_y_22 =0 
    output_x = 0
    output_y = 0

    # MDA 
    for t in range(1,T+1):
      # H = 2
        x_11, y_11, r_11 = update_xy(x_11,y_11,R,lr)
        x_12, y_12, r_12 = update_xy(x_12,y_12,R,lr)
        x_21, y_21, r_21 = update_xy(x_21,y_21,R,lr)
        x_22, y_22, r_22 = update_xy(x_22,y_22,R,lr)

      # H = 1
        Q = 0.1*R + np.array([[r_11,r_12],[r_21,r_22]])
        x,y,r = update_xy(x,y,Q,lr)
        
        alpha_t = 1/t
        output_x_11 = (1-alpha_t)*output_x_11 + alpha_t*x_11
        output_y_11 = (1-alpha_t)*output_y_11 + alpha_t*y_11
        output_x_12 = (1-alpha_t)*output_x_12 + alpha_t*x_12
        output_y_12 = (1-alpha_t)*output_y_12 + alpha_t*y_12
        output_x_21 = (1-alpha_t)*output_x_21 + alpha_t*x_21
        output_y_21 = (1-alpha_t)*output_y_21 + alpha_t*y_21
        output_x_22 = (1-alpha_t)*output_x_22 + alpha_t*x_22
        output_y_22 = (1-alpha_t)*output_y_22 + alpha_t*y_22
        output_x = (1-alpha_t)*output_x + alpha_t*x
        output_y = (1-alpha_t)*output_y + alpha_t*y
        
    data_lst.append([output_x_11,output_y_11,output_x_12,output_y_12,
                    output_x_21,output_y_21,output_x_22,output_y_22,output_x,output_y])
    NE_gap_val = NE_gap(output_x_11,output_y_11,output_x_12,output_y_12,
                    output_x_21,output_y_21,output_x_22,output_y_22,output_x,output_y,R)
    NE_gap_lst.append(NE_gap_val)
    dist_lst.append(np.sum(np.abs(output_x_11-0.5) + np.abs(output_y_11-0.5) +
                          np.abs(output_x_12-0.5) + np.abs(output_y_12-0.5) +
                           np.abs(output_x_21-0.5) + np.abs(output_y_21-0.5) +
                           np.abs(output_x_22-0.5) + np.abs(output_y_22-0.5) +
                           np.abs(output_x-0.5) + np.abs(output_y-0.5)
                          ))
    print('T = ', T, '  NE_gap:', NE_gap_val)
    if T >= 1e6-2:
        np.savez('counter_example1', T_lst_counter =T_lst, NE_gap_lst = NE_gap_lst, data_lst = data_lst,dist_lst = dist_lst)
np.savez('counter_example1', T_lst = T_lst, NE_gap_lst = NE_gap_lst, data_lst = data_lst,dist_lst = dist_lst)

T =  1000   NE_gap: 0.05659444865381069
T =  3000   NE_gap: 0.0683906798657885
T =  10000   NE_gap: 0.025733615548878763
T =  30000   NE_gap: 0.012876807253965783
T =  100000   NE_gap: 0.010935471069025704
T =  300000   NE_gap: 0.009045901486506192
T =  1000000   NE_gap: 0.005288941105574763
T =  3000000   NE_gap: 0.005072203872712899
T =  10000000   NE_gap: 0.004672689933458929


In [ ]:
# -------------------------------------- Single Trial Figure (Figure 1(b)(c)) Data -------------------------------------

In [15]:
def NE_gap_each_layer(x_11,y_11,x_12,y_12,
                    x_21,y_21,x_22,y_22,x,y,R):
    qx_11 = np.dot(R,y_11)
    qx_12 = np.dot(R,y_12)
    qx_21 = np.dot(R,y_21)
    qx_22 = np.dot(R,y_22)
    qy_11 = np.dot(np.transpose(R),x_11)
    qy_12 = np.dot(np.transpose(R),x_12)
    qy_21 = np.dot(np.transpose(R),x_21)
    qy_22 = np.dot(np.transpose(R),x_22)
    NE_gap_layer_2 = max([(np.max(qx_11) - np.min(qy_11)) ,(np.max(qx_12) - np.min(qy_12)),
                      (np.max(qx_21) - np.min(qy_21)) ,(np.max(qx_22) - np.min(qy_22))])
    
    Q_star = 0.5*np.ones([2,2]) + 0.1*R
    qx = np.dot(Q_star,y)
    qy = np.dot(np.transpose(Q_star),x)
    NE_gap_layer_1 = np.max(qx) - np.min(qy)
    return NE_gap_layer_1, NE_gap_layer_2

In [16]:
# FTRL
H = 2
R = np.identity(2)
T = 1e7
T = int(T)
lr = 1/np.sqrt(T)
NE_gap_layer_1_lst = []
NE_gap_layer_2_lst = []

Q_11 = np.ones([2,2])
Q_12 = np.ones([2,2])
Q_21 = np.ones([2,2])
Q_22 = np.ones([2,2])
Q = 2*np.ones([2,2])

qx_11 = 0#np.dot(Q_11,y_11)
qy_11 = 0#-np.dot(np.transpose(Q_11),x_11)
qx_12 = 0#np.dot(Q_12,y_12)
qy_12 = 0#-np.dot(np.transpose(Q_12),x_12)
qx_21 = 0#np.dot(Q_21,y_21)
qy_21 = 0#-np.dot(np.transpose(Q_21),x_21)
qx_22 = 0#np.dot(Q_22,y_22)
qy_22 = 0#-np.dot(np.transpose(Q_22),x_22)
qx = 0#np.dot(Q,y)
qy = 0#-np.dot(np.transpose(Q),x)
w_t = 0

output_x_11 = 0
output_y_11 = 0
output_x_12 = 0
output_y_12 = 0
output_x_21 = 0
output_y_21 = 0
output_x_22 = 0
output_y_22 =0 
output_x = 0
output_y = 0

for t in range(1,T+1):
    # Actor
    if w_t == 0:
        eta = 0
    else:
        eta = lr/w_t
    x_11 = prop_exponential(eta*qx_11 + np.log(init_x_11))
    y_11 = prop_exponential(eta*qy_11+ np.log(init_y_11))
    x_12 = prop_exponential(eta*qx_12+ np.log(init_x_12))
    y_12 = prop_exponential(eta*qy_12+ np.log(init_y_12))
    x_21 = prop_exponential(eta*qx_21+ np.log(init_x_21))
    y_21 = prop_exponential(eta*qy_21+ np.log(init_y_21))
    x_22 = prop_exponential(eta*qx_22+ np.log(init_x_22))
    y_22 = prop_exponential(eta*qy_22+ np.log(init_y_22))
    x = prop_exponential(eta*qx+ np.log(init_x))
    y = prop_exponential(eta*qy+ np.log(init_y))


    # Critic 
    alpha_t = (H+1)/(H+t) 
    if t == 1:
        alpha_t_1 = 1
    else:
        alpha_t_1 = alpha_t_1*(1-alpha_t)
    w_t = alpha_t/alpha_t_1
    r_11 = np.dot(x_11, np.dot(R,y_11))
    r_12 = np.dot(x_12, np.dot(R,y_12))
    r_21 = np.dot(x_21, np.dot(R,y_21))
    r_22 = np.dot(x_22, np.dot(R,y_22))
    Q = (1-alpha_t)*Q + alpha_t*(0.1*R + np.array([[r_11,r_12],[r_21,r_22]]))

    qx_11 = qx_11 + w_t* np.dot(R,y_11)
    qy_11 = qy_11 - w_t*np.dot(np.transpose(R),x_11)
    qx_12 = qx_12 + w_t* np.dot(R,y_12)
    qy_12 = qy_12 - w_t*np.dot(np.transpose(R),x_12)
    qx_21 = qx_21 + w_t* np.dot(R,y_21)
    qy_21 = qy_21 - w_t*np.dot(np.transpose(R),x_21)
    qx_22 = qx_22 + w_t* np.dot(R,y_22)
    qy_22 = qy_22 - w_t*np.dot(np.transpose(R),x_22)
    qx = qx + w_t*np.dot(Q,y)
    qy = qy - w_t*np.dot(np.transpose(Q),x)

    # Output policy
    output_x_11 = (1-alpha_t)*output_x_11 + alpha_t*x_11
    output_y_11 = (1-alpha_t)*output_y_11 + alpha_t*y_11
    output_x_12 = (1-alpha_t)*output_x_12 + alpha_t*x_12
    output_y_12 = (1-alpha_t)*output_y_12 + alpha_t*y_12
    output_x_21 = (1-alpha_t)*output_x_21 + alpha_t*x_21
    output_y_21 = (1-alpha_t)*output_y_21 + alpha_t*y_21
    output_x_22 = (1-alpha_t)*output_x_22 + alpha_t*x_22
    output_y_22 = (1-alpha_t)*output_y_22 + alpha_t*y_22
    output_x = (1-alpha_t)*output_x + alpha_t*x
    output_y = (1-alpha_t)*output_y + alpha_t*y
    NE_gap_layer_1, NE_gap_layer_2 = NE_gap_each_layer(output_x_11,output_y_11,output_x_12,output_y_12,
                    output_x_21,output_y_21,output_x_22,output_y_22,output_x,output_y,R)
    NE_gap_layer_1_lst.append(NE_gap_layer_1)
    NE_gap_layer_2_lst.append(NE_gap_layer_2)
np.savez('FTRL_single_trial2', NE_gap_layer_1_lst = NE_gap_layer_1_lst, NE_gap_layer_2_lst = NE_gap_layer_2_lst)

In [19]:
## Counter example
def update_xy(x,y,R,lr):
    gx = np.dot(R,y)
    gy = -np.dot(np.transpose(R),x)
    x = x * np.exp(gx*lr)
    y = y * np.exp(gy*lr)
    x = x / sum(x)
    y = y / sum(y)
    r = np.dot(x, np.dot(R,y))
    return x,y,r

NE_gap_layer_1_lst = []
NE_gap_layer_2_lst = []
R = np.identity(2)

T = 1e7
T = int(T)
lr = 1/np.sqrt(T)
# ------------------------------------------------------------------------------------------
# Running MD-critic for given the following initialization

# initialization H=1
x = init_x
y = init_y

# initialization on H=2
x_11 = init_x_11#np.array([.25,.75]) #x_slice[:,idx_max]
x_12 = init_x_12#np.array([.5,.5])#x_slice[:,idx_mid]
x_21 = init_x_21#np.array([.5,.5])#x_slice[:,idx_mid]
x_22 = init_x_22#np.array([.75,.25])#x_slice[:,idx_min]
y_11 = init_y_11#np.array([.25,.75])#y_slice[:,idx_max]
y_12 = init_y_12#np.array([.17,.83])#y_slice[:,idx_mid]
y_21 = init_y_21#np.array([.17,.83])#y_slice[:,idx_mid]
y_22 = init_y_22#np.array([.25,.75])#y_slice[:,idx_min]

output_x_11 = 0
output_y_11 = 0
output_x_12 = 0
output_y_12 = 0
output_x_21 = 0
output_y_21 = 0
output_x_22 = 0
output_y_22 =0 
output_x = 0
output_y = 0

# MDA 
for t in range(1,T+1):
  # H = 2
    x_11, y_11, r_11 = update_xy(x_11,y_11,R,lr)
    x_12, y_12, r_12 = update_xy(x_12,y_12,R,lr)
    x_21, y_21, r_21 = update_xy(x_21,y_21,R,lr)
    x_22, y_22, r_22 = update_xy(x_22,y_22,R,lr)

  # H = 1
    Q = 0.1*R + np.array([[r_11,r_12],[r_21,r_22]])
    x,y,r = update_xy(x,y,Q,lr)

    alpha_t = 1/t
    output_x_11 = (1-alpha_t)*output_x_11 + alpha_t*x_11
    output_y_11 = (1-alpha_t)*output_y_11 + alpha_t*y_11
    output_x_12 = (1-alpha_t)*output_x_12 + alpha_t*x_12
    output_y_12 = (1-alpha_t)*output_y_12 + alpha_t*y_12
    output_x_21 = (1-alpha_t)*output_x_21 + alpha_t*x_21
    output_y_21 = (1-alpha_t)*output_y_21 + alpha_t*y_21
    output_x_22 = (1-alpha_t)*output_x_22 + alpha_t*x_22
    output_y_22 = (1-alpha_t)*output_y_22 + alpha_t*y_22
    output_x = (1-alpha_t)*output_x + alpha_t*x
    output_y = (1-alpha_t)*output_y + alpha_t*y
        
    NE_gap_layer_1, NE_gap_layer_2 = NE_gap_each_layer(output_x_11,output_y_11,output_x_12,output_y_12,
                    output_x_21,output_y_21,output_x_22,output_y_22,output_x,output_y,R)
    NE_gap_layer_1_lst.append(NE_gap_layer_1)
    NE_gap_layer_2_lst.append(NE_gap_layer_2)
    
np.savez('counter_example_single_trial2', NE_gap_layer_1_lst = NE_gap_layer_1_lst, NE_gap_layer_2_lst = NE_gap_layer_2_lst)

In [ ]:
# --------------------------------------------Appendix Figures -------------------------------------------------

In [ ]:
# INPG
T = int(1e6)
lr = 1/np.sqrt(T)
Xs_INPG = np.zeros(T)
Ys_INPG = np.zeros(T)
X_11s_INPG = np.zeros(T)
Y_11s_INPG = np.zeros(T)
X_12s_INPG = np.zeros(T)
Y_12s_INPG = np.zeros(T)
X_21s_INPG = np.zeros(T)
Y_21s_INPG = np.zeros(T)
X_22s_INPG = np.zeros(T)
Y_22s_INPG = np.zeros(T)

r_11s_INPG = np.zeros(T)
r_12s_INPG = np.zeros(T)
r_21s_INPG = np.zeros(T)
r_22s_INPG = np.zeros(T)
# initialization H=1
x = init_x
y = init_y

# initialization on H=2
x_11 = init_x_11#np.array([.25,.75]) #x_slice[:,idx_max]
x_12 = init_x_12#np.array([.5,.5])#x_slice[:,idx_mid]
x_21 = init_x_21#np.array([.5,.5])#x_slice[:,idx_mid]
x_22 = init_x_22#np.array([.75,.25])#x_slice[:,idx_min]
y_11 = init_y_11#np.array([.25,.75])#y_slice[:,idx_max]
y_12 = init_y_12#np.array([.17,.83])#y_slice[:,idx_mid]
y_21 = init_y_21#np.array([.17,.83])#y_slice[:,idx_mid]
y_22 = init_y_22#np.array([.25,.75])#y_slice[:,idx_min]
R = np.identity(2)

# MDA 
for t in range(T):
  # H = 2
    x_11, y_11, r_11 = update_xy(x_11,y_11,R,lr)
    x_12, y_12, r_12 = update_xy(x_12,y_12,R,lr)
    x_21, y_21, r_21 = update_xy(x_21,y_21,R,lr)
    x_22, y_22, r_22 = update_xy(x_22,y_22,R,lr)

  # H = 1
    Q = 0.1*np.identity(2) + np.array([[r_11,r_12],[r_21,r_22]])
    x,y,r = update_xy(x,y,Q,lr)
    Xs_INPG[t] = x[0]
    Ys_INPG[t] = y[0]
    X_11s_INPG[t] = x_11[0]
    Y_11s_INPG[t] = y_11[0]
    X_12s_INPG[t] = x_12[0]
    Y_12s_INPG[t] = y_12[0]
    X_21s_INPG[t] = x_21[0]
    Y_21s_INPG[t] = y_21[0]
    X_22s_INPG[t] = x_22[0]
    Y_22s_INPG[t] = y_22[0]
    r_11s_INPG[t] = r_11
    r_12s_INPG[t] = r_12
    r_21s_INPG[t] = r_21
    r_22s_INPG[t] = r_22

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4.5))
ax.set_facecolor('whitesmoke')
ax.set_axisbelow(True)
ax.tick_params(direction='in')
plt.xlim([9.5e5, 1e6])
plt.grid(ls='dotted')
plt.plot(r_11s_INPG,label = r'$V_2^{\mu^t\!,\!\nu^t}(s_{11})$')
plt.plot(r_12s_INPG, label = r'$V_2^{\mu^t\!,\!\nu^t}(s_{12}), V_2^{\mu^t\!,\!\nu^t}(s_{21})$')
#plt.plot(r_21s_INPG[idx_lst])
plt.plot(r_22s_INPG, label = r'$V_2^{\mu^t\!,\! \nu^t}(s_{22})$')

plt.xlabel(r"Iteration step $t$", fontsize=15)
#plt.ylabel(r"$\mu_2^t(a_1|s_{11})$", fontsize=15) #$V_1^{\dagger,\widehat{\nu}^T}(s_1) \!-\!V_1^{\widehat{\mu}^T, \dagger}(s_1)$
plt.legend(bbox_to_anchor=(0., 1.01, 1., .101),loc='lower left',
                      ncol=3, mode="expand", borderaxespad=0.,fontsize=12)
plt.tight_layout()
plt.savefig('appendix1.pdf')

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4.5))
ax.set_facecolor('whitesmoke')
ax.set_axisbelow(True)
plt.xlim([6e5,1e6])
#plt.ylim([.15,.9])
ax.tick_params(direction='in')
plt.grid(ls='dotted')
plt.plot(Xs_INPG,label = r'$\mu_1^t(a_1|s_{0})$')
plt.plot(np.mean(Xs_INPG)*(np.ones(Xs_INPG.shape)), 
         label = r'state-wise average $\widehat{\mu}_1^T(a_1|s_{0})$', linestyle='dashed')
plt.xlabel(r"Iteration step $t$", fontsize=15)
plt.legend(bbox_to_anchor=(0., 1.01, 1., .101),loc='lower left',
                      ncol=2, mode="expand", borderaxespad=0.,fontsize=13)
plt.tight_layout()
plt.savefig('appendix2.pdf')

In [ ]:
# How the initializations values are found

### two-layers Mirror descent ###
## H = 2 produce cyclic values

for T in [3e7]:
    T = int(T)
    print('T = ', T, ': ------------------------')
    # --------------------------------------------------------------------
    # This section is just for finding a properly 'bad' initialization
    lr = 1/np.sqrt(T)
    values = np.zeros(T)
    x_lst = np.zeros([2,T])
    y_lst = np.zeros([2,T])

    # initialization
    x = np.array([0.2,0.8])
    y = np.array([0.4,0.6])

    # MDA 
    for t in range(T):
      R = np.identity(2)
      gx = np.dot(R,y)
      gy = -np.dot(np.transpose(R),x)
      x = x * np.exp(gx*lr)
      y = y * np.exp(gy*lr)
      x = x / sum(x)
      y = y / sum(y)
      values[t] = np.dot(x,np.dot(np.identity(2),y))
      x_lst[:,t] = x
      y_lst[:,t] = y

    # find the correct index for initialization
    # Note that we need to find the index according to the value curve instead of x or y curve! The frequencies are different!
    slice_idx = range(T-T//10,T)
    x_slice = x_lst[:,slice_idx]
    y_slice = y_lst[:,slice_idx]
    value_slice = values[slice_idx]
    idx_max = np.argmax(value_slice)
    idx_min = np.argmin(value_slice)
    idx_mid = (idx_max + idx_min)//2

    # initialization on H=2
    x_11 = x_slice[:,idx_max]
    x_12 = x_slice[:,idx_mid]
    x_21 = x_slice[:,idx_mid]
    x_22 = x_slice[:,idx_min]
    y_11 = y_slice[:,idx_max]
    y_12 = y_slice[:,idx_mid]
    y_21 = y_slice[:,idx_mid]
    y_22 = y_slice[:,idx_min]
    print('Initialization values:')
    print('x_11:', x_11[0],x_11[1], '; y_11:',y_11[0],y_11[1])
    print('x_12:', x_12[0],x_12[1], '; y_12:',y_12[0],y_12[1])
    print('x_21:', x_21[0],x_21[1], '; y_21:',y_21[0],y_21[1])
    print('x_22:', x_22[0],x_22[1], '; y_22:',y_22[0],y_22[1])